In [1]:
from math import ceil, sqrt
from typing import cast

import radon
import torch
import torchvision
import tqdm
from pathlib import Path

from lodopab_dataset import LoDoPaBDataset
from lodopab2_dataset import LoDoPaB2Dataset
from ellipses2_dataset import Ellipses2Dataset

In [2]:
img_size = 64
angles = torch.linspace(0.0, torch.pi, 257)[:-1]
positions = torch.arange(-ceil(img_size*1.41421356237/2.0), ceil(img_size*1.41421356237/2.0)+1, dtype=torch.float)

In [3]:
file_count = sum(1 for _ in Path("/data/datasets/Ellipses/train").iterdir())
for path in tqdm.notebook.tqdm(Path("/data/datasets/Ellipses/train").iterdir(), total=file_count):
    if path.name.startswith("ground_truth_"):
        gt = torch.load(str(path.resolve())).contiguous()
        torch.save(gt, f"/data/datasets/Ellipses/train/ground_truth_{path.stem[13:]}.pt")
        sino = radon.radon_forward(gt.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
        torch.save(sino, f"/data/datasets/Ellipses/train/sinogram_{path.stem[13:]}.pt")

file_count = sum(1 for _ in Path("/data/datasets/Ellipses/val").iterdir())
for path in tqdm.notebook.tqdm(Path("/data/datasets/Ellipses/val").iterdir(), total=file_count):
    if path.name.startswith("ground_truth_"):
        gt = torch.load(str(path.resolve())).contiguous()
        torch.save(gt, f"/data/datasets/Ellipses/val/ground_truth_{path.stem[13:]}.pt")
        sino = radon.radon_forward(gt.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
        torch.save(sino, f"/data/datasets/Ellipses/val/sinogram_{path.stem[13:]}.pt")

file_count = sum(1 for _ in Path("/data/datasets/Ellipses/test").iterdir())
for path in tqdm.notebook.tqdm(Path("/data/datasets/Ellipses/test").iterdir(), total=file_count):
    if path.name.startswith("ground_truth_"):
        gt = torch.load(str(path.resolve())).contiguous()
        torch.save(gt, f"/data/datasets/Ellipses/test/ground_truth_{path.stem[13:]}.pt")
        sino = radon.radon_forward(gt.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
        torch.save(sino, f"/data/datasets/Ellipses/test/sinogram_{path.stem[13:]}.pt")

  0%|          | 0/20480 [00:00<?, ?it/s]

  0%|          | 0/5120 [00:00<?, ?it/s]

  0%|          | 0/6400 [00:00<?, ?it/s]

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(img_size, antialias=cast(str, True)),
    torchvision.transforms.CenterCrop(img_size)
])


dataset = LoDoPaBDataset("/data/datasets", LoDoPaBDataset.Subset.TRAIN, extracted=True)
for i, sample in tqdm.notebook.tqdm(enumerate(dataset), total=len(dataset)):
    ground_truth = transform(sample[1])
    sinogram = radon.radon_forward(ground_truth.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
    torch.save(ground_truth, f"/data/datasets/LoDoPaB2/train/ground_truth_{i}.pt")
    torch.save(sinogram, f"/data/datasets/LoDoPaB2/train/sinogram_{i}.pt")
    if i == len(dataset)-1:
        break

dataset = LoDoPaBDataset("/data/datasets", LoDoPaBDataset.Subset.VAL, extracted=True)
for i, sample in tqdm.notebook.tqdm(enumerate(dataset), total=len(dataset)):
    ground_truth = transform(sample[1])
    sinogram = radon.radon_forward(ground_truth.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
    torch.save(ground_truth, f"/data/datasets/LoDoPaB2/val/ground_truth_{i}.pt")
    torch.save(sinogram, f"/data/datasets/LoDoPaB2/val/sinogram_{i}.pt")
    if i == len(dataset)-1:
        break

dataset = LoDoPaBDataset("/data/datasets", LoDoPaBDataset.Subset.TEST, extracted=True)
for i, sample in tqdm.notebook.tqdm(enumerate(dataset), total=len(dataset)):
    ground_truth = transform(sample[1])
    sinogram = radon.radon_forward(ground_truth.unsqueeze(0).to("cuda"), angles.to("cuda"), positions.to("cuda"))[0].to("cpu")
    torch.save(ground_truth, f"/data/datasets/LoDoPaB2/test/ground_truth_{i}.pt")
    torch.save(sinogram, f"/data/datasets/LoDoPaB2/test/sinogram_{i}.pt")
    if i == len(dataset)-1:
        break

In [ ]:
matrix = radon.radon_matrix(torch.zeros(img_size, img_size, device="cuda"), thetas=angles.to("cuda"), positions=positions.to("cuda"))
v, d, ut = torch.linalg.svd(matrix, full_matrices=False)
torch.save(v.mT.to("cpu"), "/home/alexander/Projects/convergent-data-driven-regularizations-for-ct-reconstruction/cache/vt.pt")
torch.save(d.to("cpu"), "/home/alexander/Projects/convergent-data-driven-regularizations-for-ct-reconstruction/cache/d.pt")
torch.save(ut.mT.to("cpu"), "/home/alexander/Projects/convergent-data-driven-regularizations-for-ct-reconstruction/cache/u.pt")

In [5]:
sample = Ellipses2Dataset("/data/datasets", Ellipses2Dataset.Subset.TEST)[0]
gt_e = sample[1].to("cuda")

sino_gne = sample[0].to("cuda")                                                              #gne:  0.0
sino_gle = sino_gne+0.005*torch.randn_like(sino_gne)                                         #gle:  0.014
sino_gme = sino_gne+0.015*torch.randn_like(sino_gne)                                         #gme:  0.046
sino_ghe = sino_gne+0.03*torch.randn_like(sino_gne)                                          #ghe:  0.095

sino_une = Ellipses2Dataset("/data/datasets", Ellipses2Dataset.Subset.TEST)[0][0].to("cuda") #une:  0.0
sino_ule = sino_une-sqrt(3.0)*0.005+2.0*sqrt(3.0)*0.005*torch.rand_like(sino_une)            #ule:  0.0148
sino_ume = sino_une-sqrt(3.0)*0.015+2.0*sqrt(3.0)*0.015*torch.rand_like(sino_une)            #ume:  0.047
sino_uhe = sino_une-sqrt(3.0)*0.03+2.0*sqrt(3.0)*0.03*torch.rand_like(sino_une)              #uhe:  0.095

sample = LoDoPaB2Dataset("/data/datasets", LoDoPaB2Dataset.Subset.TEST)[0]
gt_l = sample[1].to("cuda")
sino_gnl = sample[0].to("cuda")                                                              #gnl:  0.0
sino_gll = sino_gnl+0.005*torch.randn_like(sino_gnl)                                         #gll:  0.0288
sino_gml = sino_gnl+0.015*torch.randn_like(sino_gnl)                                         #gml:  0.079
sino_ghl = sino_gnl+0.03*torch.randn_like(sino_gnl)                                          #ghl:  0.13

sino_unl = LoDoPaB2Dataset("/data/datasets", LoDoPaB2Dataset.Subset.TEST)[0][0].to("cuda")   #unl:  0.0
sino_ull = sino_unl-sqrt(3.0)*0.005+2.0*sqrt(3.0)*0.005*torch.rand_like(sino_unl)            #ull:  0.028
sino_uml = sino_unl-sqrt(3.0)*0.015+2.0*sqrt(3.0)*0.015*torch.rand_like(sino_unl)            #uml:  0.074
sino_uhl = sino_unl-sqrt(3.0)*0.03+2.0*sqrt(3.0)*0.03*torch.rand_like(sino_unl)              #uhl:  0.15

sino_gxe = sino_gne+0.01*torch.randn_like(sino_gne)                                          #gxe:  0.03
sino_uxe = sino_une+0.01*torch.randn_like(sino_une)                                          #uxe:  0.033
sino_gxl = sino_gnl+0.01*torch.randn_like(sino_gnl)                                          #gxl:  0.055
sino_uxl = sino_unl+0.01*torch.randn_like(sino_unl)                                          #uxl:  0.055


for short in ["gxe","uxe","gne","gle","gme","ghe","une","ule","ume","uhe","gnl","gll","gml","ghl","unl","ull","uml","uhl","gxl","uxl"]:
    alpha = 0.0

    while True:
        in_ = input(short+"-alpha: ")
        if in_ == "q":
            break
        else:
            alpha = float(in_)

        A_ = radon.radon_matrix(torch.zeros(img_size, img_size, device="cuda"), thetas=angles.to("cuda"), positions=positions.to("cuda"))
        alphaI = alpha*torch.eye(A_.shape[1], device="cuda")
        ATA = A_.mT@A_
        A = ATA + alphaI

        sino = globals()["sino_"+short]
        b = radon.radon_backward(sino.unsqueeze(0), img_size, angles.to("cuda"), positions.to("cuda"))[0]
        b = b.reshape(*b.shape[:-2], -1, 1)
        z = torch.linalg.solve(A, b)
        recon = z.reshape(sino.shape[0], 1, img_size, img_size)
        recon_sino = radon.radon_forward(recon, thetas=angles.to("cuda"), positions=positions.to("cuda"))[0]
        loss = torch.nn.functional.mse_loss(recon_sino, sino)
        lvl = {"n": 0.0, "l": 0.005, "m": 0.015, "h": 0.03, "x": 0.01}[short[1]]
        print(f"{short}-alpha: {alpha}   {short}-loss: {loss.item():5.3e} / {lvl**2:5.3e}", flush=True)

    torch.save(A, f"/home/alexander/Projects/convergent-data-driven-regularizations-for-ct-reconstruction/cache/A_{short}.pt")

gnl-alpha: 0.0   gnl-loss: 1.673e-11 / 0.000e+00


In [ ]:
A = radon.radon_matrix(torch.zeros(img_size, img_size, device="cuda"), thetas=angles.to("cuda"), positions=positions.to("cuda"))
torch.save(A.mT@A, f"/home/alexander/Projects/convergent-data-driven-regularizations-for-ct-reconstruction/cache/A.pt")